In [30]:
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import pandas as pd
import time
import os
import dotenv
dotenv.load_dotenv()


True

In [11]:
token = os.getenv('RUNPOD_TOKEN')
open_ai_base_url = os.getenv('RUNPOD_EMBEDDING_URL')
model_name = os.getenv('MODEL_NAME')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
index_name = os.getenv('PINECONE_INDEX_NAME')

In [12]:
pc = Pinecone(api_key=pinecone_api_key)
client = OpenAI(
    api_key=token,
    base_url=open_ai_base_url
)

# Try Out Embeddings

In [13]:
output = client.embeddings.create(input=['hello world'], model=model_name)
embedding = output.data[0].embedding
print(embedding)

[0.015176702290773392, -0.02265065535902977, 0.00859504658728838, -0.0742514431476593, 0.003908572718501091, 0.0027112148236483335, -0.031238075345754623, 0.044630181044340134, 0.04405056685209274, -0.007908663712441921, -0.02519790083169937, -0.03337348997592926, 0.014375921338796616, 0.04639952629804611, 0.008610299788415432, -0.016137639060616493, 0.007561658509075642, -0.019020449370145798, -0.11458028852939606, -0.018105272203683853, 0.1262945681810379, 0.029728032648563385, 0.025274166837334633, -0.0342886708676815, -0.041091494262218475, 0.006669359747320414, 0.010303379036486149, 0.022437114268541336, 0.00444242637604475, -0.12727075815200806, -0.01610713265836239, -0.020255940034985542, 0.047375716269016266, 0.011561748571693897, 0.06815025955438614, 0.00739006232470274, -0.017998501658439636, 0.04084744676947594, -0.01028049923479557, 0.02373361587524414, 0.010509294457733631, -0.028538301587104797, 0.008137458004057407, -0.015138569287955761, 0.030948270112276077, -0.0659538

In [14]:
len(embedding)

384

# Data Wrangling

In [15]:
df = pd.read_json('products/products.jsonl', lines=True)
df.head()

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,savory_scone.jpg
2,Latte,Coffee,"Smooth and creamy, our latte combines rich esp...","[Espresso, Steamed Milk, Milk Foam]",4.75,4.8,latte.jpg
3,Chocolate Chip Biscotti,Bakery,"Crunchy and delightful, this chocolate chip bi...","[Flour, Sugar, Chocolate Chips, Eggs, Almonds,...",2.50,4.6,chocolate_biscotti.jpg
4,Espresso shot,Coffee,"A bold shot of rich espresso, our espresso is ...",[Espresso],2.00,4.9,espresso_shot.jpg


In [16]:
df['text'] = df['name'] + " : " + df["description"] +\
            " -- Ingredient: " + df['ingredients'].astype(str) +\
            " -- Price: " + df['price'].astype(str) +\
            " -- Rating: " + df['rating'].astype(str)

In [18]:
texts = df['text'].to_list()

["Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredient: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 4.5 -- Rating: 4.7",
 "Jumbo Savory Scone : Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea. -- Ingredient: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- Price: 3.25 -- Rating: 4.3",
 "Latte : Smooth and creamy, our latte combines rich espresso with velvety steamed milk, creating a perfect balance of flavor and texture. Enjoy it as a comforting treat any time of day, whether you're starting your morning or taking a midday break. -- Ingredient: ['Espresso

In [20]:
with open('products/about_us.txt') as f:
    aboutUs_section = f.read()
aboutUs_section = "Seaside Sips' About Us Section: " + aboutUs_section
texts.append(aboutUs_section)

In [21]:
with open('products/menu_items_text.txt') as f:
    menu_items_text = f.read()
menu_items_text = "Menu items: " + menu_items_text
texts.append(menu_items_text)

# Generate Embeddings

In [23]:
output = client.embeddings.create(input=texts, model=model_name)
embeddings = output.data

# Push Data to Database

In [25]:
pc.create_index(
    name=index_name,
    dimension=384,
    metric='cosine',
    spec=ServerlessSpec (
        cloud='aws',
        region='us-east-1'
    )
)

{
    "name": "seasidesip-coffeeshop",
    "metric": "cosine",
    "host": "seasidesip-coffeeshop-brjs6ay.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [33]:
# wait for index to be ready
while not pc.describe_index(index_name).status.ready:
    time.sleep(1)
index = pc.Index(index_name)

vectors = []
for text, e in zip(texts, embeddings):
    entry_id = text.split(":")[0]
    vectors.append({
        "id": entry_id,
        "values": e.embedding,
        "metadata": {"text": text}
    })
index.upsert(vectors=vectors, namespace='ns1')

{'upserted_count': 20}

# Get Closest Documents

In [34]:
output = client.embeddings.create(input=['Is Cappuccino lactose-free?'], model=model_name)
embedding = output.data[0].embedding

In [35]:
results = index.query(
    namespace='ns1',
    vector=embedding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

In [36]:
results

{'matches': [{'id': 'Cappuccino ',
              'metadata': {'text': 'Cappuccino : A rich and creamy cappuccino '
                                   'made with freshly brewed espresso, steamed '
                                   'milk, and a frothy milk cap. This '
                                   'delightful drink offers a perfect balance '
                                   'of bold coffee flavor and smooth milk, '
                                   'making it an ideal companion for relaxing '
                                   'mornings or lively conversations. -- '
                                   "Ingredient: ['Espresso', 'Steamed Milk', "
                                   "'Milk Foam'] -- Price: 4.5 -- Rating: 4.7"},
              'score': 0.735915959,
              'values': []},
             {'id': 'Sugar Free Vanilla syrup ',
              'metadata': {'text': 'Sugar Free Vanilla syrup : Enjoy the sweet '
                                   'flavor of vanilla without the